
# GRU for Stock Price Prediction

## 1. Introduction

In this tutorial, we will use a Gated Recurrent Unit (GRU) model to predict stock prices based on historical data. GRUs are particularly well-suited for sequential data like stock prices because they can capture temporal dependencies and patterns.

## 2. Stock Data

We will use stock data from Yahoo Finance, which contains information like the opening, closing, high, low prices, and volume traded over time.

Let's start by fetching the stock data.
    

In [ ]:

import pandas as pd
import numpy as np
import yfinance as yf

# Fetch stock data from Yahoo Finance
ticker = 'AAPL'
data = yf.download(ticker, start='2015-01-01', end='2023-01-01')

# Display the first few rows
data.head()
    


## 3. Preprocessing the Data

We will preprocess the data by normalizing the closing prices and creating sequences that the GRU can learn from.
    

In [ ]:

from sklearn.preprocessing import MinMaxScaler

# We will use the 'Close' column as our target variable
data = data[['Close']]

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Create sequences for the GRU model
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(seq_length, len(data)):
        X.append(data[i-seq_length:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

# Define sequence length
seq_length = 60  # Using 60 days of data to predict the next day

# Create the sequences
X, y = create_sequences(scaled_data, seq_length)

# Reshape the data for the GRU model
X = X.reshape((X.shape[0], X.shape[1], 1))

# Split the data into training and test sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]
    


## 4. Building the GRU Model

Now, let's build the GRU model using TensorFlow/Keras.
    

In [ ]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense

# Build the GRU model
model = Sequential([
    GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    GRU(units=50, return_sequences=False),
    Dense(units=25),
    Dense(units=1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
    


## 5. Evaluating the Model

Let's evaluate the model's performance on the test set.
    

In [ ]:

# Make predictions
predicted_stock_price = model.predict(X_test)

# Inverse transform to get actual stock prices
predicted_stock_price = scaler.inverse_transform(predicted_stock_price.reshape(-1, 1))
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

# Plot the results
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(y_test_actual, color='blue', label='Actual Stock Price')
plt.plot(predicted_stock_price, color='red', label='Predicted Stock Price')
plt.title(f'{ticker} Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()
    


## 6. Conclusion

In this notebook, we used a GRU model to predict stock prices based on historical data. GRUs are effective for time series forecasting tasks like this one, where the temporal dependencies in the data are important for making accurate predictions.
    